In [36]:
import dashy.models as m
import dashy.server as s
import requests
import json
import datetime as dt
import time
import numpy as np
from tqdm import tqdm_notebook

In [139]:
data = pd.read_csv("train.csv", encoding="WINDOWS-1251", sep=";",  header=None,
                   names=["date", "calls_in", "calls_proc", "calls_lost", "mean_response_time", "mean_call_time", "mean_operators", "max_operators"])
data.drop(0, inplace=True)
data.set_index("date", inplace=True)
data = data.astype(np.int)
data.index = pd.to_datetime(data.index, dayfirst=True)
data.head(5)

,calls_in,calls_proc,calls_lost,mean_response_time,mean_call_time,mean_operators,max_operators
date,,,,,,,
2017-03-01,2142,1875,264,84,123,5,10
2017-03-02,1781,1736,43,23,117,6,11
2017-03-03,1785,1653,126,49,114,5,10
2017-03-04,1073,1024,41,28,121,4,6
2017-03-05,572,559,4,8,106,3,4


In [48]:
from influxdb import InfluxDBClient
client = InfluxDBClient('localhost', 8086, 'root', 'root', 'example')

In [154]:
# Ensure DB is cleean
client.drop_measurement("calls_in")
client.drop_measurement("calls_in_model_retrain")
client.drop_measurement("calls_in_predictions")
client.drop_measurement("mean_response_time")
client.drop_measurement("calls_lost")
client.drop_measurement("calls_proc")

# Push data to our service
for datetime, x in tqdm_notebook(data.iterrows()):
    body = json.dumps({"type":"calls_in", "value": float(x['calls_in']), "datetime": str(datetime)})
    requests.post("http://127.0.0.1:5000/callcenter/api/v1.0/metric", json=body)
    body = json.dumps({"type":"calls_proc", "value": float(x['calls_proc']), "datetime": str(datetime)})
    requests.post("http://127.0.0.1:5000/callcenter/api/v1.0/metric", json=body)
    body = json.dumps({"type":"calls_lost", "value": float(x['calls_lost']), "datetime": str(datetime)})
    requests.post("http://127.0.0.1:5000/callcenter/api/v1.0/metric", json=body)
    body = json.dumps({"type":"mean_response_time", "value": float(x['mean_response_time']), "datetime": str(datetime)})
    requests.post("http://127.0.0.1:5000/callcenter/api/v1.0/metric", json=body)